In [1]:
import cv2
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk, ImageEnhance, ImageFilter
import face_recognition
import numpy as np
import threading
import time


class FaceDetectionTool:
    def __init__(self, root):
        self.root = root
        self.root.title("Advanced Face Detection Enhancement Tool")
        self.root.geometry("1920x1080")  # Full screen for 1920x1080 displays
        self.image_path = None
        self.original_image = None
        self.processed_image = None
        self.face_detected = False
        self.stop_calibration = False
        self.confidence = 0.0
        self.minimum_confidence = 80.0  # Target confidence percentage
        self.logs = []  # Stores logs of settings with highest confidence

        # Default enhancement parameters
        self.settings = {
            "brightness": 1.0,
            "sharpness": 1.0,
            "contrast": 1.0,
            "hue": 0,
            "saturation": 1.0,
            "gamma": 1.0,
            "blur": 0,
            "edge_enhancement": 0,
            "shadows": 0.0,
            "highlights": 0.0,
        }

        # Create GUI elements
        self.create_gui()

    def create_gui(self):
        # Divide the screen into sections
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_rowconfigure(1, weight=3)
        self.root.grid_rowconfigure(2, weight=2)
        self.root.grid_columnconfigure(0, weight=1)
        self.root.grid_columnconfigure(1, weight=1)

        # Image display frame
        image_frame = tk.Frame(self.root)
        image_frame.grid(row=0, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)

        self.image_label = tk.Label(image_frame, text="No Image Loaded", font=("Arial", 14), bg="gray", fg="white")
        self.image_label.pack(fill="both", expand=True)

        # Controls frame
        controls_frame = tk.Frame(self.root)
        controls_frame.grid(row=1, column=0, sticky="nsew", padx=10, pady=10)

        self.status_label = tk.Label(controls_frame, text="Load an image to start.", font=("Arial", 14))
        self.status_label.pack(pady=10)

        self.confidence_label = tk.Label(controls_frame, text="Confidence: 0%", font=("Arial", 12))
        self.confidence_label.pack(pady=5)

        button_frame = tk.Frame(controls_frame)
        button_frame.pack(pady=10)
        tk.Button(button_frame, text="Load Image", command=self.load_image).pack(side="left", padx=5)
        tk.Button(button_frame, text="Copy Settings", command=self.copy_settings).pack(side="left", padx=5)
        tk.Button(button_frame, text="Save Processed Image", command=self.save_image).pack(side="left", padx=5)
        self.calibrate_button = tk.Button(button_frame, text="Start Calibration", command=self.start_calibration)
        self.calibrate_button.pack(side="left", padx=5)

        # Settings sliders frame
        sliders_frame = tk.Frame(self.root)
        sliders_frame.grid(row=1, column=1, sticky="nsew", padx=10, pady=10)

        canvas = tk.Canvas(sliders_frame)
        scrollbar = ttk.Scrollbar(sliders_frame, orient="vertical", command=canvas.yview)
        self.scrollable_sliders = tk.Frame(canvas)

        self.scrollable_sliders.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all")),
        )

        canvas.create_window((0, 0), window=self.scrollable_sliders, anchor="n")
        canvas.configure(yscrollcommand=scrollbar.set)

        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        # Create sliders for settings
        for setting, default in self.settings.items():
            frame = tk.Frame(self.scrollable_sliders)
            frame.pack(pady=10, anchor="center", fill="x")

            tk.Label(frame, text=setting.capitalize()).pack(side="top", pady=5)
            slider = tk.Scale(
                frame,
                from_=0 if setting != "gamma" else 0.1,
                to=2 if setting != "hue" else 180,
                resolution=0.1,
                orient="horizontal",
                command=lambda val, s=setting: self.update_setting(s, val),
                length=700,
            )
            slider.set(default)
            slider.pack()

        # Logs frame
        logs_frame = tk.Frame(self.root)
        logs_frame.grid(row=2, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)

        tk.Label(logs_frame, text="Logged Settings (Most Confident)", font=("Arial", 12)).pack()

        logs_canvas = tk.Canvas(logs_frame)
        logs_scrollbar = ttk.Scrollbar(logs_frame, orient="vertical", command=logs_canvas.yview)
        self.log_buttons_frame = tk.Frame(logs_canvas)

        self.log_buttons_frame.bind(
            "<Configure>",
            lambda e: logs_canvas.configure(scrollregion=logs_canvas.bbox("all")),
        )

        logs_canvas.create_window((0, 0), window=self.log_buttons_frame, anchor="n")
        logs_canvas.configure(yscrollcommand=logs_scrollbar.set)

        logs_canvas.pack(side="left", fill="both", expand=True)
        logs_scrollbar.pack(side="right", fill="y")

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.png;*.jpeg")])
        if not file_path:
            return
        self.image_path = file_path
        image = cv2.imread(self.image_path)
        self.original_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.original_image = self.resize_image(self.original_image)
        self.processed_image = self.original_image.copy()
        self.apply_enhancements()

    def resize_image(self, image, max_width=800, max_height=600):
        height, width = image.shape[:2]
        scaling_factor = min(max_width / width, max_height / height)
        new_width = int(width * scaling_factor)
        new_height = int(height * scaling_factor)
        return cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    def display_image(self, image, face_locations=None):
        img = image.copy()
        if face_locations:
            for top, right, bottom, left in face_locations:
                cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)
        img = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(img)
        self.image_label.imgtk = imgtk
        self.image_label.configure(image=imgtk, text="")

    def apply_enhancements(self):
        if self.original_image is None:
            return
        pil_image = Image.fromarray(self.original_image)

        # Apply enhancements
        pil_image = ImageEnhance.Brightness(pil_image).enhance(self.settings["brightness"])
        pil_image = ImageEnhance.Sharpness(pil_image).enhance(self.settings["sharpness"])
        pil_image = ImageEnhance.Contrast(pil_image).enhance(self.settings["contrast"])
        pil_image = ImageEnhance.Color(pil_image).enhance(self.settings["saturation"])

        # Detect faces and calculate confidence
        self.processed_image = np.array(pil_image)
        face_locations = face_recognition.face_locations(self.processed_image)
        self.update_confidence(face_locations)

        # Update GUI
        self.display_image(self.processed_image, face_locations)

    def update_confidence(self, face_locations):
        """
        Confidence is calculated based on the proportion of the image covered by detected faces.
        """
        if not face_locations:
            self.confidence = 0.0
            self.status_label.config(text="No Face Detected", fg="red")
        else:
            total_area = self.processed_image.shape[0] * self.processed_image.shape[1]
            face_area = sum((bottom - top) * (right - left) for top, right, bottom, left in face_locations)
            self.confidence = (face_area / total_area) * 100
            self.status_label.config(text=f"Face(s) Detected: {len(face_locations)}", fg="green")

        self.confidence_label.config(text=f"Confidence: {self.confidence:.2f}%")

    def update_setting(self, setting, value):
        self.settings[setting] = float(value)
        self.apply_enhancements()

    def copy_settings(self):
        settings = ", ".join(f"{key}: {value}" for key, value in self.settings.items())
        self.root.clipboard_clear()
        self.root.clipboard_append(settings)
        self.root.update()
        messagebox.showinfo("Settings Copied", "Settings copied to clipboard!")

    def save_image(self):
        if self.processed_image is None:
            messagebox.showerror("Error", "No processed image to save.")
            return
        save_path = "processed_image.jpg"
        cv2.imwrite(save_path, cv2.cvtColor(self.processed_image, cv2.COLOR_RGB2BGR))
        messagebox.showinfo("Image Saved", f"Image saved to {save_path}")

    def start_calibration(self):
        self.stop_calibration = False
        self.calibrate_button.config(text="Stop Calibration", command=self.stop_calibration_process)
        threading.Thread(target=self.calibration_process).start()

    def stop_calibration_process(self):
        self.stop_calibration = True
        self.calibrate_button.config(text="Start Calibration", command=self.start_calibration)
        for button in self.log_buttons_frame.winfo_children():
            button.config(state=tk.NORMAL)

    def calibration_process(self):
        # Settings to calibrate and their step sizes
        parameters = [
            "brightness",
            "sharpness",
            "contrast",
            "hue",
            "saturation",
            "gamma",
            "blur",
            "edge_enhancement",
            "shadows",
            "highlights",
        ]
        step_sizes = {
            "brightness": 0.2,
            "sharpness": 0.2,
            "contrast": 0.2,
            "hue": 10,
            "saturation": 0.2,
            "gamma": 0.2,
            "blur": 1,
            "edge_enhancement": 1,
            "shadows": 0.2,
            "highlights": 0.2,
        }

        highest_confidence = self.confidence  # Track the highest confidence achieved

        # Generate all combinations of adjustments
        for brightness in np.arange(0.5, 2.1, step_sizes["brightness"]):
            for contrast in np.arange(0.5, 2.1, step_sizes["contrast"]):
                for sharpness in np.arange(0.5, 2.1, step_sizes["sharpness"]):
                    for hue in range(0, 180, step_sizes["hue"]):
                        for saturation in np.arange(0.5, 2.1, step_sizes["saturation"]):
                            # Update settings for this combination
                            self.settings.update({
                                "brightness": brightness,
                                "contrast": contrast,
                                "sharpness": sharpness,
                                "hue": hue,
                                "saturation": saturation,
                            })

                            self.apply_enhancements()  # Apply current combination

                            # Log the settings if confidence improves or equals the highest
                            if self.confidence >= highest_confidence:
                                if self.confidence > highest_confidence:
                                    highest_confidence = self.confidence
                                self.log_settings(self.confidence, self.settings.copy())

                            if self.stop_calibration:  # Exit if calibration is stopped
                                self.stop_calibration_process()
                                return

        # After calibration, stop and reset
        self.stop_calibration_process()

    def log_settings(self, confidence, settings):
        # Avoid logging duplicate settings
        if any(log[1] == settings for log in self.logs):
            return

        self.logs.append((confidence, settings.copy()))

        # Create and add button for the logged settings
        button = tk.Button(
            self.log_buttons_frame,
            text=f"Confidence: {confidence:.2f}%",
            command=lambda s=settings: self.apply_logged_settings(s),
            state=tk.DISABLED if self.stop_calibration is False else tk.NORMAL,
        )
        button.pack(fill=tk.X, pady=5)

    def apply_logged_settings(self, settings):
        self.settings = settings.copy()
        self.apply_enhancements()


if __name__ == "__main__":
    root = tk.Tk()
    app = FaceDetectionTool(root)
    root.mainloop()
